# Exercise 1: t-SNE

## Do not start the exercise until you fully understand the submission guidelines.


* The homework assignments are executed automatically.
* Failure to comply with the following instructions will result in a significant penalty.
* Appeals regarding your failure to read these instructions will be denied.

## Read the following instructions carefully:

1. This Jupyter notebook contains all the step-by-step instructions needed for this exercise.
1. Write **efficient**, **vectorized** code whenever possible. Some calculations in this exercise may take several minutes when implemented efficiently, and might take much longer otherwise. Unnecessary loops will result in point deductions.
1. You are responsible for the correctness of your code and should add as many tests as you see fit to this jupyter notebook. Tests will not be graded nor checked.
1. You are allowed to use functions and methods from the [Python Standard Library](https://docs.python.org/3/library/).
1. Your code must run without errors. Use at least `numpy` 1.15.4. Any code that cannot run will not be graded.
1. Write your own code. Cheating will not be tolerated.
1. Submission includes a zip file that contains this notebook, with your ID as the file name. For example, `hw1_123456789_987654321.zip` if you submitted in pairs and `hw1_123456789.zip` if you submitted the exercise alone. The name of the notebook should follow the same structure.
   
Please use only a **zip** file in your submission.

---
##❗❗❗❗❗❗❗❗❗**This is mandatory**❗❗❗❗❗❗❗❗❗
## Please write your RUNI emails in this cell:

### *** YOUR EMAILS HERE ***
---

## Please sign that you have read and understood the instructions:

### *** YOUR IDS HERE ***

---


In [4]:
# Import necessary libraries
import numpy as np
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from numpy.typing import NDArray

np.random.seed(42)

# Design your algorithm
Make sure to describe the algorithm, its limitations, and describe use-cases.

# Your implementations
You may add new cells, write helper functions or test code as you see fit.
Please use the cell below and include a description of your implementation.
Explain code design consideration, algorithmic choices and any other details you think is relevant to understanding your implementation.
Failing to explain your code will lead to point deductions.

In [33]:
class CustomTSNE:
    def __init__(
        self, perplexity=30.0, n_components=2, n_iter=1000, learning_rate=200.0
    ):
        self.perplexity = perplexity
        self.n_components = n_components
        self.n_iter = n_iter
        self.learning_rate = learning_rate
        # Note: You may add more attributes

    def _pairwise_distances(self, X: NDArray[np.float64]) -> NDArray[np.float64]:
        """
        Compute pairwise squared Euclidean distances between all samples in X.

        Parameters
        ----------
        X : ndarray of shape (n_samples, n_features)
            Input data matrix where each row represents one data point.

        Returns
        -------
        D : ndarray of shape (n_samples, n_samples)
            Matrix of squared Euclidean distances.
        """
        # Ensure floating-point type for numerical stability
        X = X.astype(np.float64)

        # ||x_i - x_j||^2 = (x_i·x_i) + (x_j·x_j) − 2*(x_i·x_j)
        sum_X = np.sum(np.square(X), axis=1)
        D = np.add.outer(sum_X, sum_X) - 2 * np.dot(X, X.T)

        # Set diagonal to 0 to avoid small numerical residuals
        np.fill_diagonal(D, 0.0)

        return D

    def _binary_search_sigma(
        self,
        distances_i: np.ndarray,
        target_perplexity: float,
        tol: float = 1e-5,
        max_iter: int = 50,
    ) -> tuple[np.ndarray, float]:
        """
        Find the Gaussian width σ_i for one data point so that its conditional
        probability distribution p_{j|i} has the desired perplexity.

        Parameters
        ----------
        distances_i : ndarray of shape (n_samples,)
            Squared distances from the i-th point to all other points.
        target_perplexity : float
            Desired perplexity (effective number of neighbors).
        tol : float, default=1e-5
            Stopping tolerance for perplexity difference.
        max_iter : int, default=50
            Maximum number of binary-search iterations.

        Returns
        -------
        p_i : ndarray of shape (n_samples,)
            Conditional probabilities p_{j|i} (sum to 1, p_{i|i}=0).
        sigma : float
            Optimal σ_i value that achieved the target perplexity.
        """
        # β = 1 / (2σ²)
        beta_min, beta_max = -np.inf, np.inf
        beta = 1.0

        # Ensure float type and ignore self-distance
        distances_i = distances_i.astype(np.float64).copy()
        distances_i[distances_i == 0] = np.inf

        for _ in range(max_iter):
            # Compute Gaussian affinities
            P_i = np.exp(-distances_i * beta)
            P_i[distances_i == np.inf] = 0.0
            sum_P_i = np.sum(P_i)
            if sum_P_i == 0:
                sum_P_i = 1e-12
            P_i /= sum_P_i

            # Compute Shannon entropy and perplexity
            H = -np.sum(P_i * np.log2(P_i + 1e-10))
            perplexity = 2**H

            # Check for convergence
            if abs(perplexity - target_perplexity) < tol:
                break

            # Adjust β (inverse variance) based on whether entropy too high/low
            if perplexity > target_perplexity:
                beta_min = beta
                beta = beta * 2 if beta_max == np.inf else (beta + beta_max) / 2
            else:
                beta_max = beta
                beta = beta / 2 if beta_min == -np.inf else (beta + beta_min) / 2

        sigma = np.sqrt(1 / (2 * beta))
        return P_i, sigma

    def _compute_P(self, X: np.ndarray) -> np.ndarray:
        """
        Compute the symmetric joint probability matrix P from high-dimensional data X.

        Parameters
        ----------
        X : ndarray of shape (n_samples, n_features)
            Input data matrix.

        Returns
        -------
        P : ndarray of shape (n_samples, n_samples)
            Symmetric joint probability matrix where P[i, j] reflects similarity
            between points i and j in high-dimensional space.
        """
        N = X.shape[0]

        # Edge case: only one point
        if N == 1:
            return np.zeros((1, 1))

        # Compute squared distances (ensure float type)
        distances = self._pairwise_distances(X).astype(np.float64)
        P = np.zeros((N, N))
        sigmas = np.zeros(N)

        # Compute conditional probabilities for each data point
        for i in range(N):
            # Handle identical points (all distances ≈ 0)
            if np.allclose(distances[i], 0):
                P[i] = np.zeros(N)
                continue
            p_i, sigma_i = self._binary_search_sigma(distances[i], self.perplexity)
            P[i] = p_i
            sigmas[i] = sigma_i

        # Symmetrize and normalize
        P = (P + P.T) / (2 * N)
        np.fill_diagonal(P, 0.0)

        # Prevent underflow and re-normalize
        P = np.maximum(P, 1e-12)
        P /= np.sum(P)

        return P

    def _compute_Q(self, Y: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
        """
        Compute low-dimensional joint probability matrix Q using Student-t distribution.

        Parameters
        ----------
        Y : ndarray of shape (n_samples, n_components)
            Current low-dimensional embedding.

        Returns
        -------
        Q : ndarray of shape (n_samples, n_samples)
            Symmetric joint probability matrix in low-dimensional space.
        num : ndarray of shape (n_samples, n_samples)
            Unnormalized pairwise affinities (for gradient reuse).
        """
        # Edge case: single point
        if Y.shape[0] == 1:
            return np.zeros((1, 1)), np.zeros((1, 1))

        # Compute pairwise squared distances
        sum_Y = np.sum(np.square(Y), axis=1)
        D = np.add.outer(sum_Y, sum_Y) - 2 * np.dot(Y, Y.T)

        # Student-t affinities
        num = 1 / (1 + D)
        np.fill_diagonal(num, 0.0)

        # Normalize
        Q = num / np.sum(num)
        Q = np.maximum(Q, 1e-12)

        return Q, num
    
    def _gradient(self, P: np.ndarray, Q: np.ndarray, Y: np.ndarray, num: np.ndarray) -> np.ndarray:
        """
        Compute the gradient of the KL divergence loss with respect to Y.

        Parameters
        ----------
        P : ndarray of shape (n_samples, n_samples)
            High-dimensional joint probabilities.
        Q : ndarray of shape (n_samples, n_samples)
            Low-dimensional joint probabilities.
        Y : ndarray of shape (n_samples, n_components)
            Current low-dimensional embedding.
        num : ndarray of shape (n_samples, n_samples)
            Unnormalized affinities (1 / (1 + ||y_i - y_j||^2)), reused for efficiency.

        Returns
        -------
        dY : ndarray of shape (n_samples, n_components)
            Gradient of the KL divergence with respect to Y.
        """
        PQ = P - Q  # difference between high-D and low-D similarities
        # (P - Q) * num gives the force magnitude per pair
        forces = 4 * (PQ * num)  # scalar weight for each pair

        # Compute gradient as matrix operation
        dY = np.zeros_like(Y)
        for i in range(Y.shape[0]):
            # weighted sum of differences (y_i - y_j)
            dY[i, :] = np.sum(np.expand_dims(forces[i, :], 1) * (Y[i, :] - Y), axis=0)

        return dY

    def fit_transform(self, X: np.ndarray) -> np.ndarray:
        """
        Fit t-SNE on high-dimensional data X and return low-dimensional embedding Y.

        Parameters
        ----------
        X : ndarray of shape (n_samples, n_features)
            Input high-dimensional data.

        Returns
        -------
        Y : ndarray of shape (n_samples, n_components)
            Low-dimensional t-SNE embedding.
        """
        N = X.shape[0]

        # Edge case: all points identical → no embedding needed
        if np.allclose(X, X[0]):
            return np.zeros((N, self.n_components))

        # Step 1: Compute high-dimensional similarities P
        P = self._compute_P(X)

        # Step 2: Initialize Y deterministically for reproducibility
        rng = np.random.default_rng(42)
        Y = rng.normal(0, 1e-4, size=(N, self.n_components))

        # Step 3: Early exaggeration (standard = ×4)
        early_exaggeration = 4.0
        P = np.maximum(P * early_exaggeration, 1e-12)

        # Step 4: Learning-rate heuristics (like sklearn’s “auto”)
        if isinstance(self.learning_rate, str) and self.learning_rate == "auto":
            self.learning_rate = max(N / early_exaggeration / 4, 50)

        # Momentum configuration
        momentum = 0.5
        final_momentum = 0.8
        momentum_switch_iter = 250
        dY_prev = np.zeros_like(Y)

        prev_kl = np.inf

        # Step 5: Gradient-descent loop
        for it in range(self.n_iter):
            Q, num = self._compute_Q(Y)
            grad = self._gradient(P, Q, Y, num)

            # Gradient update with momentum
            dY = momentum * dY_prev - self.learning_rate * grad
            Y += dY
            dY_prev = dY

            # Clip to prevent runaway values
            Y = np.clip(Y, -50, 50)

            # Compute KL divergence for monitoring
            kl = np.sum(P * np.log((P + 1e-12) / (Q + 1e-12)))

            # Reduce learning rate if divergence explodes
            if kl > prev_kl * 4:
                self.learning_rate *= 0.5
            prev_kl = kl

            # Switch momentum after warm-up phase
            if it == momentum_switch_iter:
                momentum = final_momentum

            # Reduce early exaggeration after 100 iters
            if it == 100:
                P /= early_exaggeration

            # Optional progress logging
            if (it + 1) % 100 == 0:
                print(f"Iteration {it + 1}: KL divergence = {kl:.5f}")

        return Y

    # Part 2: Transformation of New Data Points
    def transform(self, X_original, Y_original, X_new):
        # Implement your method for incorporating new points into the existing t-SNE layout
        # Your code here

        # Return Y_new, the transformed data
        pass

# Unit Testing

In [36]:
import unittest
import numpy as np

class TestCustomTSNE(unittest.TestCase):
    def setUp(self):
        """Small sample matrix for testing."""
        self.tsne = CustomTSNE()
        self.X = np.array([[0, 0], [3, 4]])
    
    def test_pairwise_distances(self):
        """Check if pairwise distance matrix is correct."""
        D = self.tsne._pairwise_distances(self.X)
        expected = np.array([[0., 25.], [25., 0.]])
        self.assertTrue(np.allclose(D, expected), "Pairwise distance computation failed")
    
    def test_binary_search_sigma(self):
        """Check if computed perplexity is close to target."""
        D = np.array([0., 1., 4., 9.])  # distances from one point to 3 others
        p_i, sigma = self.tsne._binary_search_sigma(D, target_perplexity=2.0)
        
        # Compute achieved perplexity
        H = -np.sum(p_i * np.log2(p_i + 1e-10))
        perplexity = 2 ** H
        
        self.assertAlmostEqual(perplexity, 2.0, delta=0.1, msg="Perplexity not matched")
        self.assertAlmostEqual(np.sum(p_i), 1.0, delta=1e-6, msg="Probabilities not normalized")
    
    def test_compute_P(self):
        """Check if the P matrix is valid and normalized."""
        P = self.tsne._compute_P(self.X)
        
        # P must be symmetric
        self.assertTrue(np.allclose(P, P.T, atol=1e-8), "P is not symmetric")
        # Sum of all probabilities should be 1
        self.assertAlmostEqual(np.sum(P), 1.0, delta=1e-6, msg="P not normalized")
        # Diagonal should be 0
        self.assertTrue(np.allclose(np.diag(P), 0.0), "Diagonal of P should be 0")
        # All probabilities should be positive
        self.assertTrue(np.all(P >= 0), "P contains negative values")
    
    def test_compute_P_single_point(self):
        """Edge case: single data point should yield all zeros."""
        X_single = np.array([[0.0, 0.0]])
        P = self.tsne._compute_P(X_single)
        self.assertTrue(np.allclose(P, np.zeros_like(P)), "Single point should return zeros")

    def test_compute_P_identical_points(self):
        """Edge case: identical points should not produce NaN or inf."""
        X_identical = np.array([[1.0, 1.0], [1.0, 1.0]])
        P = self.tsne._compute_P(X_identical)
        self.assertFalse(np.any(np.isnan(P)), "P contains NaNs")
        self.assertFalse(np.any(np.isinf(P)), "P contains infinities")
        self.assertAlmostEqual(np.sum(P), 1.0, delta=1e-6, msg="P not normalized")

    def test_compute_P_large_values(self):
        """Edge case: large feature magnitudes should not overflow."""
        X_large = np.array([[1000, 1000], [1003, 1004], [1006, 1008]])
        P = self.tsne._compute_P(X_large)
        self.assertTrue(np.all(P >= 0), "P contains negative values")
        self.assertAlmostEqual(np.sum(P), 1.0, delta=1e-6, msg="P not normalized")

    def test_compute_P_various_perplexities(self):
        """Ensure stability across different perplexity values."""
        for p in [2, 5, 10]:
            self.tsne.perplexity = p
            P = self.tsne._compute_P(self.X)
            self.assertAlmostEqual(np.sum(P), 1.0, delta=1e-6, msg=f"P not normalized for perplexity={p}")
            self.assertTrue(np.allclose(P, P.T, atol=1e-8), f"P not symmetric for perplexity={p}")
    
    def test_compute_Q(self):
        """Check if the Q matrix in low-dimensional space is valid and normalized."""
        # Simple 3-point 2D embedding
        Y = np.array([[0.0, 0.0], [1.0, 0.0], [0.0, 1.0]])
        Q, num = self.tsne._compute_Q(Y)
        
        # Q should be symmetric
        self.assertTrue(np.allclose(Q, Q.T, atol=1e-8), "Q is not symmetric")
        # Diagonal should be zero
        self.assertTrue(np.allclose(np.diag(Q), 0.0), "Diagonal of Q should be 0")
        # All probabilities should be positive
        self.assertTrue(np.all(Q > 0), "Q contains non-positive values")
        # Q should be normalized
        self.assertAlmostEqual(np.sum(Q), 1.0, delta=1e-6, msg="Q not normalized")

    def test_compute_Q_single_point(self):
        """Edge case: single point should yield all zeros."""
        Y = np.array([[0.0, 0.0]])
        Q, num = self.tsne._compute_Q(Y)
        self.assertTrue(np.allclose(Q, np.zeros_like(Q)), "Single point Q should be zeros")

    def test_compute_Q_identical_points(self):
        """Edge case: identical points should not cause NaN or inf."""
        Y = np.array([[1.0, 1.0], [1.0, 1.0]])
        Q, num = self.tsne._compute_Q(Y)
        self.assertFalse(np.any(np.isnan(Q)), "Q contains NaNs")
        self.assertFalse(np.any(np.isinf(Q)), "Q contains infinities")
        self.assertAlmostEqual(np.sum(Q), 1.0, delta=1e-6, msg="Q not normalized")

    def test_compute_Q_large_values(self):
        """Edge case: large coordinate magnitudes should not overflow."""
        Y = np.array([[1000, 1000], [1003, 1004], [1006, 1008]])
        Q, num = self.tsne._compute_Q(Y)
        self.assertTrue(np.all(Q >= 0), "Q contains negative values")
        self.assertAlmostEqual(np.sum(Q), 1.0, delta=1e-6, msg="Q not normalized")
    
    def test_gradient_basic(self):
        """Check that gradient has correct shape and no NaNs."""
        X = np.array([[0, 0], [3, 4], [6, 8]])
        P = self.tsne._compute_P(X)
        Y = np.random.randn(3, 2) * 0.01
        Q, num = self.tsne._compute_Q(Y)
        grad = self.tsne._gradient(P, Q, Y, num)

        # Check shape
        self.assertEqual(grad.shape, Y.shape, "Gradient shape mismatch")
        # Check finite values
        self.assertFalse(np.any(np.isnan(grad)), "Gradient contains NaN values")
        self.assertFalse(np.any(np.isinf(grad)), "Gradient contains Inf values")

    def test_gradient_zero_when_P_equals_Q(self):
        """Gradient should be near zero when P == Q."""
        Y = np.random.randn(3, 2) * 0.01
        Q, num = self.tsne._compute_Q(Y)
        P = Q.copy()
        grad = self.tsne._gradient(P, Q, Y, num)
        self.assertTrue(np.allclose(grad, 0, atol=1e-6), "Gradient should vanish when P=Q")

    def test_gradient_single_point(self):
        """Edge case: single point should yield zero gradient."""
        P = np.zeros((1, 1))
        Q = np.zeros((1, 1))
        num = np.zeros((1, 1))
        Y = np.array([[0.0, 0.0]])
        grad = self.tsne._gradient(P, Q, Y, num)
        self.assertTrue(np.allclose(grad, np.zeros_like(Y)), "Single point gradient should be zero")

    def test_gradient_identical_points(self):
        """Edge case: identical points should produce finite gradients."""
        X = np.array([[1.0, 1.0], [1.0, 1.0]])
        P = self.tsne._compute_P(X)
        Y = np.array([[0.0, 0.0], [0.0, 0.0]])
        Q, num = self.tsne._compute_Q(Y)
        grad = self.tsne._gradient(P, Q, Y, num)
        self.assertFalse(np.any(np.isnan(grad)), "Gradient contains NaNs for identical points")
        self.assertFalse(np.any(np.isinf(grad)), "Gradient contains infs for identical points")

    def test_gradient_large_values(self):
        """Edge case: large coordinates should not overflow."""
        X = np.array([[1000, 1000], [1003, 1004], [1006, 1008]])
        P = self.tsne._compute_P(X)
        Y = np.array([[1000.0, 1000.0], [1003.0, 1004.0], [1006.0, 1008.0]])
        Q, num = self.tsne._compute_Q(Y)
        grad = self.tsne._gradient(P, Q, Y, num)
        self.assertTrue(np.all(np.isfinite(grad)), "Gradient overflow for large values")
    
    def test_fit_transform_shape(self):
        """Check output shape matches (n_samples, n_components)."""
        X = np.array([[0, 0], [3, 4], [6, 8]])
        Y = self.tsne.fit_transform(X)
        self.assertEqual(Y.shape, (X.shape[0], self.tsne.n_components), "Output shape mismatch")

    def test_fit_transform_convergence_trend(self):
        """Check that KL divergence generally decreases overall."""
        X = np.random.randn(10, 5)
        tsne = CustomTSNE(n_iter=300, perplexity=5)

        # Compute initial KL divergence
        P = tsne._compute_P(X)
        Y = np.random.randn(X.shape[0], tsne.n_components) * 1e-4
        Q, _ = tsne._compute_Q(Y)
        initial_kl = np.sum(P * np.log((P + 1e-12) / (Q + 1e-12)))

        # Train normally
        Y_final = tsne.fit_transform(X)
        Q_final, _ = tsne._compute_Q(Y_final)
        final_kl = np.sum(P * np.log((P + 1e-12) / (Q_final + 1e-12)))

        # Assert only overall improvement
        self.assertLess(final_kl, initial_kl, "Final KL divergence did not decrease overall")

    
    def test_fit_transform_global_kl_reduction(self):
        """Ensure the final KL divergence is much lower than initial (global convergence)."""
        X = np.random.randn(20, 5)
        tsne = CustomTSNE(n_iter=300, perplexity=5)
        
        # Compute initial KL divergence (with random small Y)
        P = tsne._compute_P(X)
        Y_init = np.random.randn(X.shape[0], tsne.n_components) * 1e-4
        Q_init, _ = tsne._compute_Q(Y_init)
        initial_kl = np.sum(P * np.log((P + 1e-12) / (Q_init + 1e-12)))
        
        # Run the actual embedding
        Y_final = tsne.fit_transform(X)
        Q_final, _ = tsne._compute_Q(Y_final)
        final_kl = np.sum(P * np.log((P + 1e-12) / (Q_final + 1e-12)))
        
        # Assert large reduction in divergence
        self.assertLess(final_kl, initial_kl * 0.5, "KL divergence not significantly reduced")

    def test_fit_transform_single_point(self):
        """Edge case: single data point should yield zero embedding."""
        X = np.array([[0.0, 0.0]])
        Y = self.tsne.fit_transform(X)
        self.assertTrue(np.allclose(Y, np.zeros_like(Y), atol=1e-6), "Single point embedding should be zero")

    def test_fit_transform_identical_points(self):
        """Edge case: identical points should not diverge."""
        X = np.array([[1.0, 1.0], [1.0, 1.0]])
        Y = self.tsne.fit_transform(X)
        self.assertFalse(np.any(np.isnan(Y)), "Embedding contains NaNs for identical points")
        self.assertFalse(np.any(np.isinf(Y)), "Embedding contains infs for identical points")
        self.assertLess(np.max(np.abs(Y)), 1.0, "Identical points embedding diverged")

    def test_fit_transform_stability_large_values(self):
        """Edge case: large-magnitude input should not overflow."""
        X = np.array([[1000, 1000], [1003, 1004], [1006, 1008]])
        Y = self.tsne.fit_transform(X)
        self.assertTrue(np.all(np.isfinite(Y)), "Embedding overflow for large input values")

    def test_fit_transform_small_vs_large_perplexity(self):
        """Ensure results differ across very small vs. large perplexity."""
        X = np.random.randn(20, 4)
        tsne_small = CustomTSNE(perplexity=5, n_iter=300)
        tsne_large = CustomTSNE(perplexity=50, n_iter=300)
        Y_small = tsne_small.fit_transform(X)
        Y_large = tsne_large.fit_transform(X)
        self.assertFalse(np.allclose(Y_small, Y_large, atol=1e-3), "Perplexity has no effect on result")

    def test_fit_transform_reproducibility(self):
        """Ensure deterministic output for same random seed."""
        np.random.seed(42)
        X = np.random.randn(8, 4)
        tsne1 = CustomTSNE(n_iter=200)
        tsne2 = CustomTSNE(n_iter=200)
        Y1 = tsne1.fit_transform(X)
        np.random.seed(42)
        Y2 = tsne2.fit_transform(X)
        self.assertTrue(np.allclose(Y1, Y2, atol=1e-6), "Results not reproducible with same seed")




# Run all tests in the notebook
unittest.TextTestRunner().run(unittest.defaultTestLoader.loadTestsFromTestCase(TestCustomTSNE))

..................

Iteration 100: KL divergence = 11.00003
Iteration 200: KL divergence = 0.36281
Iteration 300: KL divergence = 0.12922
Iteration 100: KL divergence = 20.09445
Iteration 200: KL divergence = 0.57472
Iteration 300: KL divergence = 0.31651
Iteration 100: KL divergence = 10.60991
Iteration 200: KL divergence = 0.20323
Iteration 100: KL divergence = 10.60991
Iteration 200: KL divergence = 0.20323
Iteration 100: KL divergence = 8.48824
Iteration 200: KL divergence = -0.00000
Iteration 300: KL divergence = -0.00000
Iteration 400: KL divergence = -0.00000
Iteration 500: KL divergence = -0.00000
Iteration 600: KL divergence = -0.00000
Iteration 700: KL divergence = -0.00000
Iteration 800: KL divergence = -0.00000
Iteration 900: KL divergence = -0.00000
Iteration 1000: KL divergence = -0.00000
Iteration 100: KL divergence = 12.79132
Iteration 200: KL divergence = 1.04135
Iteration 300: KL divergence = 0.67337
Iteration 100: KL divergence = 8.98821
Iteration 200: KL divergence = 0.42365
Iteration 

......
----------------------------------------------------------------------
Ran 24 tests in 0.218s

OK


<unittest.runner.TextTestResult run=24 errors=0 failures=0>

# Load data
Please use the cell below to discuss your dataset choice and why it is appropriate (or not) for this algorithm.

In [ ]:
# Load data

# Normalize data if necessary

# Split the data into train and test

# t-SNE demonstration
Demonstrate your t-SNE implementation.

Add plots and figures. The code below is just to help you get started, and should not be your final submission.

Please use the cell below to describe your results and tests.

Describe the difference between your implementation and the sklearn implementation. Hint: you can look at the documentation.

In [ ]:
# Run your custom t-SNE implementation
custom_tsne = CustomTSNE(n_components=2, perplexity=N/10)
custom_Y = custom_tsne.fit_transform(X_train)

# Run sklearn t-SNE
sk_tsne = TSNE(n_components=2, init='random', perplexity=N/10)
sk_Y = sk_tsne.fit_transform(X_train)

# Visualization of the result
plt.figure()
plt.scatter(custom_Y[:, 0], custom_Y[:, 1], s=5, c=label_train.astype(int), cmap='tab10')
plt.scatter(custom_Y[:, 0], custom_Y[:, 1], s=5, c=label_train.astype(int), cmap='tab10')
plt.colorbar()
plt.title('MNIST Data Embedded into 2D with Custom t-SNE')

plt.figure()
plt.scatter(sk_Y[:, 0], sk_Y[:, 1], s=5, c=label_train.astype(int), cmap='tab10')
plt.colorbar()
plt.title('MNIST Data Embedded into 2D with sklearn t-SNE')
plt.show()

# t-SNE extension - mapping new samples
Demonstrate your t-SNE transformation procedure.

Add plots and figures.

Please use the cell below t describe your suggested approach in detail. Use formal notations where appropriate.
Describe and discuss your results.

In [ ]:
# Transform new data
custom_Y_new = custom_tsne.transform(X_train,custom_Y,X_test)

# Visualization of the result
plt.figure()
plt.scatter(custom_Y[:, 0], custom_Y[:, 1], s=5, c=label_train.astype(int), cmap='tab10')
plt.scatter(custom_Y_new[:, 0], custom_Y_new[:, 1], marker = '*', s=50, linewidths=0.5, edgecolors='k', c=label_test.astype(int), cmap='tab10')
plt.colorbar()
plt.title('MNIST Data Embedded into 2D with Custom t-SNE')

# Use of generative AI
Please use the cell below to describe your use of generative AI in this assignment.